# Getting Started with the Tavily API

Welcome to this tutorial on using the Tavily API! Tavily is a powerful search API designed specifically for large language models (LLMs) and AI agents. It goes beyond traditional search engines by not just finding information but also by understanding and organizing it in a way that's easy for AI applications to use. 

In this notebook, we'll explore the key features of the Tavily Python SDK, including:

* **Basic and Advanced Search:** Performing simple and in-depth web searches.
* **Search Parameters:** Customizing searches to get the most relevant results.
* **Content Extraction:** Getting the full content from a webpage.
* **Q&A Search:** Getting direct answers to your questions.

## 1. Setup

First, let's install the necessary libraries and set up our environment.

In [2]:
#%pip install tavily-python python-dotenv -q

Next, you'll need a Tavily API key. You can get a free key from the [Tavily website](https://tavily.com/). Once you have your key, create a file named `.env` in the same directory as this notebook and add your API key to it like this:

```
TAVILY_API_KEY=your_api_key_here
```

Now, we can load the API key into our environment.

In [3]:
import os
from dotenv import load_dotenv
from tavily import TavilyClient

load_dotenv()

tavily_api_key = os.getenv("TAVILY_API_KEY")
if not tavily_api_key:
    raise ValueError("TAVILY_API_KEY not found in .env file")

tavily_client = TavilyClient(api_key=tavily_api_key)

## 2. Basic Search

Let's start with a simple search. The `search` method is the most common way to use the Tavily API. It takes a query and returns a list of relevant search results.

In [4]:
response = tavily_client.search(query="What are the latest advancements in AI?")
for result in response["results"]:
    print(f"- {result['title']}: {result['url']}")

- Advancements in AI and Machine Learning: https://ep.jhu.edu/news/advancements-in-ai-and-machine-learning/
- Top AI Trends 2025: Key Developments to Watch - Appinventiv: https://appinventiv.com/blog/ai-trends/
- Latest AI Breakthroughs and News: June, July, August 2025: https://www.crescendo.ai/news/latest-ai-news-and-updates
- 2024: A year of extraordinary progress and advancement in AI: https://blog.google/technology/ai/2024-ai-extraordinary-progress-advancement/
- AI News | Latest AI News, Analysis & Events: https://www.artificialintelligence-news.com/


## 3. Advanced Search Options

Tavily offers several parameters to help you refine your search and get more specific results.

### `search_depth`

You can control the depth of the search with the `search_depth` parameter. 
- `basic`: A faster search that returns a summary and a list of sources.
- `advanced`: A more comprehensive search that also includes a detailed analysis and can answer more complex questions.

In [5]:
advanced_response = tavily_client.search(
    query="What are the long-term implications of quantum computing on cybersecurity?",
    search_depth="advanced"
)
print(advanced_response)

{'query': 'What are the long-term implications of quantum computing on cybersecurity?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://thequantuminsider.com/2025/02/01/quantum-computing-is-a-long-term-cybersecurity-risk-but-deserves-immediate-attention-analysts-report/', 'title': 'Quantum Computing is a Long-Term Cybersecurity Risk, But ...', 'content': 'The report, aimed mainly at the Intelligence Community (IC) and written by MITRE researchers Yaakov Weinstein and Brandon Rodenburg, assesses the state of quantum computing and its implications for national security. The primary concern is that once a sufficiently powerful quantum computer exists, it could render today’s encryption obsolete. The researchers predict that an RSA-2048 encryption key — currently used to secure classified information — will remain safe for at least the next few [...] Resources\n       Education\n       Reports\n       Featured\n       Insights\n       Interviews\n   

### `topic`

You can also specify a `topic` for your search to get more relevant results. The available topics are `general`, `news`, and `finance`.

In [6]:
news_response = tavily_client.search(
    query="Latest news on renewable energy",
    topic="news"
)
for result in news_response["results"]:
    print(f"- {result['title']}")

- Picking winners - The River Reporter
- Managing new political risk for renewable energy projects - Utility Dive
- Botswana: Aterian Secures Three New Mineral Prospecting Licenses - Energy Capital & Power
- ReNew Reports Strong Q1 FY26 Results with Significant Growth in Capacity, Revenue, and Profitability - SolarQuarter
- Renewable power purchase agreement prices rising in wake of One Big Beautiful Bill Act - Utility Dive


### Other Search Parameters

Here are a few other useful parameters:

* `max_results`: The maximum number of results to return (default is 5).
* `include_domains` and `exclude_domains`: A list of domains to include or exclude from the search.
* `time_range`: Filter results by time. Options are `day`, `week`, `month`, and `year`.
* `include_answer`: If `True`, the response will include a direct answer to the query.
* `include_images`: If `True`, the response will include a list of relevant images.

In [7]:
custom_search_response = tavily_client.search(
    query="What is the future of electric vehicles?",
    max_results=3,
    include_domains=["forbes.com", "wired.com"],
    time_range="month",
    include_answer=True,
    include_images=True
)

if "answer" in custom_search_response:
    print("Answer:", custom_search_response["answer"])

if "images" in custom_search_response:
    print("\nImages:")
    for image_url in custom_search_response["images"]:
        print(f"- {image_url}")

Answer: The electric vehicle market is growing rapidly, with China leading in sales and the US projected to reach $95.9 billion in revenue. Despite some manufacturers canceling EV plans, competition in electric racing is intensifying with new, advanced models. The future of EVs looks promising with continued innovation and market expansion.

Images:
- https://media.wired.com/photos/67fe89375a1c9da890c8dec8/191:100/w_1280,c_limit/LM250408_GPD-0616.jpg?mbid=social_retweet
- https://media.wired.com/photos/667aa9dcc5773adbcb812280/master/w_2560,c_limit/VERNE%20Exterior5.jpg
- https://media.wired.com/photos/67dc368ff2993b487f5302cd/3:2/w_2560,c_limit/tesla-gear-2203866493.jpg
- https://media.wired.com/photos/63dd96cc2c04f953ea757121/master/w_1600,c_limit/Xpeng-G9-Exterior-Charging-Gear.jpg
- https://media.wired.com/photos/6568fb31224df4ba72121e10/191:100/w_1280,c_limit/Tesla-Cybertruck-Delivery-Event-Roundup-Business.jpg?mbid=social_retweet


## 4. Content Extraction

Sometimes, you need more than just a snippet of information. With `extract`, you can get the full content of a webpage. This is especially useful when you need to perform a deeper analysis of the information.

In [8]:
# First, get a URL from a search result
search_response = tavily_client.search(query="What is reinforcement learning?")
first_result_url = search_response["results"][0]["url"]

# Now, extract the content
extracted_content = tavily_client.extract(urls=[first_result_url])
print(extracted_content)

{'results': [{'url': 'https://online.york.ac.uk/resources/what-is-reinforcement-learning/', 'raw_content': '[Skip to Content](#contentarea)\n\n# What is reinforcement learning?\n\nReinforcement learning (RL) is a subset of machine learning that allows an AI-driven system (sometimes referred to as an agent) to learn through trial and error using feedback from its actions. This feedback is either negative or positive, signalled as punishment or reward with, of course, the aim of maximising the reward function. RL learns from its mistakes and offers[artificial intelligence](https://venturebeat.com/2021/06/09/deepmind-says-reinforcement-learning-is-enough-to-reach-general-ai/)\xa0that mimics natural intelligence as closely as it is currently possible.\n\nIn terms of learning methods, RL is similar to supervised learning only in that it uses mapping between input and output, but that is the only thing they have in common. Whereas in supervised learning, the feedback contains the correct set

## 5. Q&A Search

If you just need a quick, direct answer to a question, the `qna_search` method is perfect. It's optimized to return a concise answer without the extra details of a full search.

In [9]:
qna_response = tavily_client.qna_search(query="Who won the 2023 Nobel Prize in Physics?")
print(qna_response)

The 2023 Nobel Prize in Physics was awarded to Pierre Agostini, Ferenc Krausz, and Anne L’Huillier for their work on attosecond laser pulses. These scientists developed methods to generate extremely short pulses of light to study electron dynamics.


## Conclusion

You now have a solid understanding of how to use the Tavily API to perform powerful and customized web searches. You've learned how to:

* Set up your environment and API key.
* Perform basic and advanced searches.
* Use various search parameters to refine your results.
* Extract the full content of webpages.
* Get quick answers with Q&A search.

For more in-depth information, you can always refer to the [official Tavily documentation](https://docs.tavily.com/). Happy searching!